# scPrinter setup for Bos taurus
Our goal? Being to characterize differential TF profiles between peak and late lactation

## Setup

In [1]:
%load_ext autoreload
%autoreload 2
import scprinter as scp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
import os
import pickle
import torch
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
from scanpy.plotting.palettes import zeileis_28
from tqdm.contrib.concurrent import *
from tqdm.auto import *
import anndata
import scanpy as sc
import json
import csv
import re
from sklearn.preprocessing import OneHotEncoder

/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. 

In [2]:
work_dir="/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man"
genome_dir="/net/talisker/home/benos/mae117/.local/share/genomes/ARS-UCD2.0"
frags_dir_controls="/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/data/controls/fragments-str"
frags_dir_peaks="/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/data/experiments/peak/fragments-str"
frags_dir_late="/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/data/experiments/late/fragments-str"

In [3]:
controls_files=os.listdir(frags_dir_controls)
controls_files_frags=[i for i in controls_files if i.endswith(".fragments.tsv.gz")]
controls_files_frags=sorted([os.path.join(frags_dir_controls, i) for i in controls_files_frags])
controls_files_samples=["_".join(re.split("[/_\\.]", i)[11:13]) for i in controls_files_frags]

In [4]:
peak_files=os.listdir(frags_dir_peaks)
peak_files_frags=[i for i in peak_files if i.endswith(".fragments.tsv.gz")]
peak_files_frags=sorted([os.path.join(frags_dir_peaks, i) for i in peak_files_frags])
peak_files_samples=["_".join(re.split("[/_\\.]", i)[11:13]) for i in peak_files_frags]

In [5]:
late_files=os.listdir(frags_dir_late)
late_files_frags=[i for i in late_files if i.endswith(".fragments.tsv.gz")]
late_files_frags=sorted([os.path.join(frags_dir_late, i) for i in late_files_frags])
late_files_samples=["_".join(re.split("[/_\\.]", i)[11:13]) for i in late_files_frags]

## Create a custom ARS-UCD2.0 for scPrinter to work on!

### We need to build a [custom genome object](https://ruochiz.com/scprinter_doc/tutorials/custom_genome_tutorial.html) for cows RE scPrinter


* Read in the chromosome sizes of the cow genome as a dictionary

In [6]:
#os.mkdir(os.path.join(work_dir, "data/custom_genomes/ARS-UCD2.0"))

# sizes have to be read in like a dictionary
chrom_sizes = {}
#with open(os.path.join(genome_dir, "ARS-UCD2.0.fa.sizes")) as f:
with open(os.path.join(genome_dir, "bosTau9.chrom.sizes")) as f:
    for line in f:
       (key, val) = line.split()
       chrom_sizes[key] = int(val)

* Create a bias file for ARDS-UCD2

In [7]:
pretrain_Tn5_bias_model = scp.datasets.pretrained_Tn5_bias_model
pretrain_Tn5_bias_model

'/net/talisker/home/benos/mae117/.cache/scprinter/Tn5_NN_model_py_v2.pt'

In [8]:
scp.genome.predict_genome_tn5_bias(fa_file=str(os.path.join(genome_dir, "ARS-UCD2.0.fa")),
                            save_name=f"{work_dir}/data/bias.h5",
                            tn5_model=pretrain_Tn5_bias_model,
                            context_radius=50,
                            device="cuda",
                            batch_size=5000)

Predicting Tn5 bias:   0%|          | 0/1958 [00:00<?, ?it/s]

* Create the custom genome

In [9]:
def make_btau_splits(with_chr_prefix=True, include_mt=False):
    """
    Create five 'reasonable' ARS-UCD2.0 chromosome splits (train/valid/test) in the
    same structure as the human example. Partitions are disjoint and exhaustive.

    Args:
        with_chr_prefix (bool): if True, chromosomes are named 'chr1'...'chr29','chrX','chrY' (and 'chrMT' if included)
                                if False, they are '1'...'29','X','Y' ('MT' if included)
        include_mt (bool): include mitochondrial chromosome in the partitions.

    Returns:
        List[Dict[str, List[str]]]: five dicts with keys 'train', 'valid', 'test'
    """

    # ARS-UCD2.0 autosomes 1..29 plus sex chromosomes
    autosomes = [f"chr{str(i)}" for i in range(1, 30)]
    sex = ["chrX", "chrY"]
    mt = ["MT"] if include_mt else []
    all_chroms = autosomes + sex + mt

    # Helper to add/remove 'chr' prefix
    def lab(c):
        #return f"chr{c}" if with_chr_prefix else c
        return f"{c}" if with_chr_prefix else c

    # ----
    # Pre-chosen, balanced-ish folds (mix large+medium+small autosomes, spread X/Y across folds)
    # These are *chromosome labels without prefix*; we’ll map to prefixed labels at the end.
    # Each fold lists its 'test' and 'valid' sets; 'train' is derived as set difference of all_chroms.

    folds_spec = [
        # Fold 1
        {
            "test":  ["chr1", "chr3", "chr6", "chr12", "chr24"],
            "valid": ["chr8", "chr20"],
        },
        # Fold 2
        {
            "test":  ["chr2", "chr9", "chr16", "chr21", "chr27"],
            "valid": ["chr12", "chr17"],
        },
        # Fold 3
        {
            "test":  ["chr4", "chr11", "chr15", "chr19", "chr29", "chrY"],
            "valid": ["chr7", "chr22"],
        },
        # Fold 4
        {
            "test":  ["chr5", "chr10", "chr14", "chr18", "chr20", "chr26"],
            "valid": ["chr6", "chr21"],
        },
        # Fold 5
        {
            "test":  ["chr7", "chr13", "chr17", "chr23", "chr25", "chrX"],
            "valid": ["chr10", "chr18"],
        },
    ]

    # Optionally distribute MT (if included), put it into validation of Fold 1
    if include_mt:
        folds_spec[0]["valid"] = folds_spec[0]["valid"] + ["MT"]

    # Build full folds, enforcing disjoint/exhaustive partition per fold
    folds = []
    all_set = set(all_chroms)
    for spec in folds_spec:
        test = set(spec["test"])
        valid = set(spec["valid"])
        # Sanity: ensure declared sets exist
        assert test.isdisjoint(valid), "test and valid overlap in spec"
        assert test.issubset(all_set) and valid.issubset(all_set), "spec includes unknown chromosomes"
        train = sorted(all_set - test - valid, key=lambda x: (x not in sex+mt, x))  # stable-ish sort

        fold = {
            "test":  sorted([lab(c) for c in test],  key=lambda x: (x not in {lab("X"), lab("Y"), lab("MT")}, x)),
            "valid": sorted([lab(c) for c in valid], key=lambda x: (x not in {lab("X"), lab("Y"), lab("MT")}, x)),
            "train": sorted([lab(c) for c in train], key=lambda x: (x not in {lab("X"), lab("Y"), lab("MT")}, x)),
        }
        folds.append(fold)

    return folds

btau_splits = make_btau_splits(with_chr_prefix=True, include_mt=False)
btau_splits

[{'test': ['chr1', 'chr12', 'chr24', 'chr3', 'chr6'],
  'valid': ['chr20', 'chr8'],
  'train': ['chr10',
   'chr11',
   'chr13',
   'chr14',
   'chr15',
   'chr16',
   'chr17',
   'chr18',
   'chr19',
   'chr2',
   'chr21',
   'chr22',
   'chr23',
   'chr25',
   'chr26',
   'chr27',
   'chr28',
   'chr29',
   'chr4',
   'chr5',
   'chr7',
   'chr9',
   'chrX',
   'chrY']},
 {'test': ['chr16', 'chr2', 'chr21', 'chr27', 'chr9'],
  'valid': ['chr12', 'chr17'],
  'train': ['chr1',
   'chr10',
   'chr11',
   'chr13',
   'chr14',
   'chr15',
   'chr18',
   'chr19',
   'chr20',
   'chr22',
   'chr23',
   'chr24',
   'chr25',
   'chr26',
   'chr28',
   'chr29',
   'chr3',
   'chr4',
   'chr5',
   'chr6',
   'chr7',
   'chr8',
   'chrX',
   'chrY']},
 {'test': ['chr11', 'chr15', 'chr19', 'chr29', 'chr4', 'chrY'],
  'valid': ['chr22', 'chr7'],
  'train': ['chr1',
   'chr10',
   'chr12',
   'chr13',
   'chr14',
   'chr16',
   'chr17',
   'chr18',
   'chr2',
   'chr20',
   'chr21',
   'chr23',
   

In [10]:
momoo = scp.genome.Genome(
    name=str(os.path.join(work_dir, "data/custom_genomes/ARS-UCD2.0")),
    chrom_sizes=chrom_sizes,
    gff_file=f"{genome_dir}/ARS-UCD2.0.annotation.gtf",
    fa_file=f"{genome_dir}/ARS-UCD2.0.fa",
    bias_file=f"{work_dir}/data/bias.h5",
    blacklist_file=f"{genome_dir}/ARS-UCD2.0.blacklist.bed", # empty bed file
    bg=None,
    splits=btau_splits
)

In [11]:
import pickle
pickle.dump(momoo, open(f"{work_dir}/data/custom_genomes/ARS-UCD2.0", "wb"))

## Create scPrinter object(s)

In [12]:
objects = []
with (open(f"{work_dir}/data/custom_genomes/ARS-UCD2.0", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

momoo=objects[0]
momoo

In [13]:
exp_frags=peak_files_frags+late_files_frags
exp_frags

['/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/data/experiments/peak/fragments-str/SRR33155817.fragments.tsv.gz',
 '/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/data/experiments/peak/fragments-str/SRR33155819.fragments.tsv.gz',
 '/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/data/experiments/peak/fragments-str/SRR33155821.fragments.tsv.gz',
 '/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/data/experiments/peak/fragments-str/SRR33155823.fragments.tsv.gz',
 '/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/data/experiments/late/fragments-str/SRR33155816.fragments.tsv.gz',
 '/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/data/experiments/late/fragments-str/SRR33155818.fragments.tsv.gz',
 '/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/data/experiments/late/fragments-str/SRR33155820.fragments.tsv.gz',
 '/net/talisker/home/benos/

In [16]:
printer_exp=scp.pp.import_fragments(
    path_to_frags=exp_frags,
    barcodes=[None] * len(exp_frags),
    savename=os.path.join(work_dir, 'profiles/ars-ucd_exp_scprinter.h5ad'),
    genome=momoo,
    min_num_fragments=1000, min_tsse=7,
    sorted_by_barcode=False,
    low_memory=False,

    # Added
    auto_detect_shift=False,    # do NOT run the shape-sensitive auto-detect routine
    plus_shift=4,               # canonical Tn5 offset for ATAC (forward strand)
    minus_shift=-5,             # canonical Tn5 offset for ATAC (reverse strand),
)

Multiple fragments files detected, it is suggested to provide sample names to avoid barcode collision


Importing fragments:   0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. 

start transferring insertions


In [17]:
pfs=[f+f"_{i}" for i, f in enumerate(peak_files_samples)]
lfs=[f+f"_{i}" for i, f in enumerate(late_files_samples)]
samples=pfs+lfs
samples

['experiments_peak_0',
 'experiments_peak_1',
 'experiments_peak_2',
 'experiments_peak_3',
 'experiments_late_0',
 'experiments_late_1',
 'experiments_late_2',
 'experiments_late_3']

In [18]:
print (printer_exp.insertion_file.obs_names[:])

['SRR33155823', 'SRR33155816', 'SRR33155817', 'SRR33155821', 'SRR33155819', 'SRR33155818', 'SRR33155820', 'SRR33155822']


In [19]:
printer_exp.insertion_file.obs_names = samples
printer_exp.insertion_file.obs_names

['experiments_peak_0',
 'experiments_peak_1',
 'experiments_peak_2',
 'experiments_peak_3',
 'experiments_late_0',
 'experiments_late_1',
 'experiments_late_2',
 'experiments_late_3']

## Call peaks on the fragments in a preset for training seq2PRINT model

In [20]:
# import tempfile
# import shutil
# import subprocess

# print(subprocess.check_output(["macs2", "--version"]).decode())

# # Create a temporary bin dir and symlink macs2 → macs3
# tmp_bin = tempfile.mkdtemp()
# macs3_path = shutil.which("macs3")
# os.symlink(macs3_path, os.path.join(tmp_bin, "macs2"))

# # Prepend to PATH so that scPrinter and subprocesses pick it up
# os.environ["PATH"] = f"{tmp_bin}:{os.environ['PATH']}"

# print(subprocess.check_output(["macs2", "--version"]).decode())

In [21]:
# Call peaks, this set of peaks are recommended to train seq2PRINT model
scp.pp.call_peaks(
    printer=printer_exp,
    frag_file=exp_frags,
    cell_grouping=[None], # here we call peaks on the cells that are included in the final analyses
    group_names=['all'],
    preset='seq2PRINT',
    overwrite=False
)

# Fetched the cleaned peaks, save, it will be used in the next step
cleaned_peaks=pd.DataFrame(printer_exp.uns["peak_calling"]['all_cleaned'][:])
cleaned_peaks.to_csv(f'{work_dir}/peaks/seq2print_exp_narrowPeak.bed', sep='\t', header=False, index=False)

continue


/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/scprinter/preprocessing.py:842: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  peak = resize_bed_df(pd.read_csv(peak, sep="\t", header=None, comment="#"), peak_width)


Reading in peak summit file(s):
NOTE: Assuming all start coordinates are 0-based ..

Padding peak summits by: 500 bp on either side for
Removing peaks overlapping with blacklisted regions and out of bound peaks based on chromosome sizes ..



/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/scprinter/peak.py:50: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  dt = pd.read_table(file, header=None)


Filtering overlapping peaks based on peak summit score ..


ValueError: No objects to concatenate

In [ ]:
# Call peaks using chromvar preset, this set of peak are recommended to be use as cell x peak for scATAC-seq data, or analysis
scp.pp.call_peaks(
    printer=printer_late,
    frag_file=late_files_frags,
    cell_grouping=[None], # here we call peaks on the cells that are included in the final analyses
    group_names=['chromvar_all'],
    preset='chromvar',
    overwrite=False
)

# Fetched the cleaned peaks, save, it will be used in the next step
cleaned_peaks = pd.DataFrame(printer_late.uns["peak_calling"]['chromvar_late_cleaned'][:])
cleaned_peaks.to_csv(f'{work_dir}/peaks/chromvar_late_regions.bed', sep='\t', header=False, index=False)

* compare the two sets of peaks with the different presets

In [ ]:
print (pd.DataFrame(printer_late.uns["peak_calling"]['all_cleaned'][:]))

In [ ]:
print (pd.DataFrame(printer_late.uns["peak_calling"]['chromvar_all_cleaned'][:]))